# Parsing Text     
Taking acquired data and breaking it down into smaller components; analyzing a sentence by its parts and then describing the syntactic roles among those elements. 

Planning and Sequence: 

1. Convert text to all lower-case for normalcy and consistency.
2. Remove any accented characters, non-ASCII (American Standard Code for Information Interchange) characters.
3. Remove special characters. (clarification listed below) 
4. Stem or lemmatize the words. (notes below)
5. Remove stopwords. (notes below)
6. Store the clean text and the original text for use in future notebooks.

+ "Special characters include all printable characters that are neither letters nor numbers. These include punctuation or technical, mathematical characters. ASCII also includes the space (a non-visible but printable character), and therefore, does not belong to the control characters category, as one might suspect." - [source](https://www.ionos.com/digitalguide/server/know-how/ascii-codes-overview-of-all-characters-on-the-ascii-table/)

+ "Stemming and Lemmatization both generate the foundation sort of the inflected words and therefore the only difference is that stem may not be an actual word whereas, lemma is an actual language word. Stemming follows an algorithm with steps to perform on the words which makes it faster." - [source](https://towardsdatascience.com/stemming-vs-lemmatization-2daddabcb221)

+ Stop-words are words which don't have strong, meaningful connotations; for instance, ‘and’, ‘a’, ‘it's’, ‘they’. Articles, prepositions, pronouns, conjunctions, and the like.  Although useful in natural language, computational inference from these words is weak; at least for the time being. 

In [1]:
import pandas as pd
import unicodedata
import re
import json
import acquire
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

***
## Exercises

The end result of this exercise should be a file named prepare.py that defines the requested functions.

In this exercise we will be defining some functions to prepare textual data. These functions should apply equally well to both the codeup blog articles and the news articles that were previously acquired.

1. Define a function named basic_clean. It should take in a string and apply some basic text cleaning to it:

- Lowercase everything
- Normalize unicode characters
- Replace anything that is not a letter, number, whitespace or a single quote.

2. Define a function named tokenize. It should take in a string and tokenize all the words in the string.

3. Define a function named stem. It should accept some text and return the text after applying stemming to all the words.

4. Define a function named lemmatize. It should accept some text and return the text after applying lemmatization to each word.

5. Define a function named remove_stopwords. It should accept some text and return the text after removing all the stopwords.

- This function should define two optional parameters, extra_words and exclude_words. 
    - These parameters should define any additional stop words to include, and any words that we don't want to remove.

6. Use your data from the acquire to produce a dataframe of the news articles. Name the dataframe news_df.

7. Make another dataframe for the Codeup blog posts. Name the dataframe codeup_df.

8. For each dataframe, produce the following columns:

- title to hold the title
- original to hold the original article/post content
- clean to hold the normalized and tokenized original with the stopwords removed.
- stemmed to hold the stemmed version of the cleaned data.
- lemmatized to hold the lemmatized version of the cleaned data.

9. Ask yourself:

- If your corpus is 493KB, would you prefer to use stemmed or lemmatized text?
- If your corpus is 25MB, would you prefer to use stemmed or lemmatized text?
- If your corpus is 200TB of text and you're charged by the megabyte for your hosted computational resources, would you prefer to use stemmed or lemmatized text?

***

#### 1. Define a function named basic_clean.      
It should take in a string and apply some basic text cleaning to it:

- Lowercase everything
- Normalize unicode characters
- Replace anything that is not a letter, number, whitespace or a single quote.

In [18]:
def basic_clean(string):
    """
    receives a string, processes it and then returns its normalized version.
    Normalization via standard NKFD unicode, fed into an asii encoder and
    decoded back into UTF-8.
    """
    # .lower converts all alphabetic characters to lower-case
    string = string.lower()
    # Return the normal form for the Unicode string.
    # Return a utf-8 encoded version of the string in ascii.
    # If an error occurs, ignores the unencodable unicode from the result.
    # decodes the string using the codec registered for encoding: utf-8.
    string = unicodedata.normalize('NFKD', string)\
    .encode('ascii', 'ignore')\
    .decode('utf-8')
    # Return the string obtained by replacing the leftmost non-overlapping occurrences of pattern in string by the replacement. 
    string = re.sub(r"[^a-z0-9'\s]", '', string)
    return string

# I could also include .lower in the final line, hence removing a line.
# e.g., string = re.sub(r"[^a-z0-9'\s]", '', string).lower()

In [16]:
basic_clean('"Inth3MID*le0fth$N!ghtdolphinogrande"')

'inth3midle0fthnghtdolphinogrande'

*** 
#### 2. Define a function named tokenize.      
It should take in a string and tokenize all the words in the string.

In [19]:
def tokenize(string):
    """
    This function takes in a string and
    returns it in its tokenized form.  
    """
    # Create the tokenizer
    # The tok-tok tokenizer is a simple, general tokenizer, where the input has 
    # one sentence per line; thus only the final period is tokenized.
    tokenizer = nltk.tokenize.ToktokTokenizer()
    # Use the tokenizer's tokenization to the inputted string, ensure it returns as a string.
    tokenized_string = tokenizer.tokenize(string, return_str = True)
    return tokenized_string

***

#### 3. Define a function named stem.      
It should accept some text and return the text after applying stemming to all the words.

In [20]:
def stem(text):
    """
    Stemmers remove morphological affixes from words, leaving only the word stem.
    As such, this UDF is applied on a word-by-word basis, deciding what the conjugation
    is doing which aspect should be cut off. Looping or list-comprehension is 
    needed to handle multiple texts in a single executable line, and is included here.
    """
    # Create porter stemmer.
    ps = nltk.porter.PorterStemmer()
    # use list-comprehension +> stem each word for the words inside the entire document
    # split by the default, which is a single space. 
    stems = [ps.stem(word) for word in text.split()]
    # glue it back together with spaces, as it was before. 
    # Join our lists of words into a string again; assign to a variable to save changes
    text_stemmed = ' '.join(stems)
    return text_stemmed

***
#### 4. Define a function named lemmatize.       
It should accept some text and return the text after applying lemmatization to each word.

In [21]:
def lemmatize(text):
    """
    Lemmatization in linguistics is the process of grouping together the inflected forms
    of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form.
    This UDF algorithmically processes string and determine the lemma of a word based on its intended meaning.
    It then returns the lemmas.
    """
    # Create the Lemmatizer.
    wnl = nltk.stem.WordNetLemmatizer()
    # use list-comprehension to lemmatize every word.
    # string.split() => output a list of every token in the document
    lemmas = [wnl.lemmatize(word) for word in text.split()]
    # Join our list of words into a string again; assign to a variable to save alterations.
    text_lemmatized = ' '.join(lemmas)
    
    return text_lemmatized

*** 
#### 5. Define a function named remove_stopwords.      
It should accept some text and return the text after removing all the stopwords.

- This function should define two optional parameters, extra_words and exclude_words. 
    - These parameters should define any additional stop words to include, and any words that we don't want to remove.

In [8]:
def remove_stopwords(text, extra_words = [], exclude_words = []):
    '''
    This function takes in some text, optional extra_words and exclude_words parameters
    with default empty lists and returns the text after removing all stop words.
    An alternate version of this can be created/used without the inclusion of casting to set-type. 
    (A lot is going on under the hood here, and so code-commenting is extensive.) 
    '''
    # Create stopword_list by assigning our stopwords from nltk as a list variable. 
    stopword_list = stopwords.words('english')
    
    # Remove 'exclude_words' from stopword_list to keep these in my text.
    # casting as a set-type, removes the order, takes away duplication 
    # (which can be unfavorable if repeated information is desired), and 
    # captures the values in {} instead of []
    stopword_list = set(stopword_list) - set(exclude_words)
    
    # Utilizing set casting, add in 'extra_words' to stopword_list
    # .union is like .append, and is the go-to option for interacting with set
    # .union establishes that every element should be unique. More technically correct and efficient. 
    stopword_list = stopword_list.union(set(extra_words))
    
    # Split document by spaces. 
    words = text.split()
    
    # Create a list of words from my string with stopwords removed and assign to variable.
    # Every word in our document, IFF that word is not in our stopwords.
    filtered_words = [word for word in words if word not in stopword_list]
    
    # Join words in the list back into strings and assign to a variable.
    # Re-glued with spaces; the beginning is the end is the beginning. 
    string_sans_stopwords = ' '.join(filtered_words)
    # Gimmee shelter. 
    return string_sans_stopwords
    

***
#### 6. Use your data from the acquire to produce a dataframe of the news articles.      
Name the dataframe news_df.


In [7]:
news_df = acquire.get_news_articles()

In [11]:
news_df

,category,title,content,author,published
0,business,Rupee hits all-time low of 77.42 against US dollar,"The Indian rupee fell to an all-time low of 77.42 against the US dollar on Monday, Reuters reported. Asian markets were lower on Monday as US stock futures fell on fears of more policy tightening from the Federal Reserve and strict lockdown in Shanghai impacting global growth, according to Reuters.",Apaar Sharma,2022-05-09T05:05:31.000Z
1,business,"Bitcoin falls to the lowest level since January, trades below $34,000","Bitcoin fell on Monday to as low as $33,266 in morning trade, nearing January's low of $32,951 as slumping equity markets continued to hurt cryptocurrencies. It then steadied to trade above $33,600. According to BBC, the world's largest cryptocurrency has fallen by 50% since its peak in November 2021.",Pragya Swastik,2022-05-09T09:20:34.000Z
2,business,Rupee closes at all-time low of 77.50 against US dollar,"The Indian rupee weakened further on Monday to close at a new all-time low of 77.50 against the US dollar, 60 paise over its previous close. During the trading session, the rupee touched its lifetime low of 77.52. The currency was weighed down by elevated crude oil prices and a widening trade deficit.",Pragya Swastik,2022-05-09T15:27:43.000Z
3,business,Made best possible decision: IndiGo on barring differently-abled child from flight,"IndiGo's CEO Ronojoy Dutta said the airline made ""the best possible decision"" by barring a differently-abled teenager and his family from boarding a Ranchi-Hyderabad flight. ""At boarding area, the teenager was visibly in panic...the airport staff, in line with safety guidelines, were forced to make a difficult decision,"" Dutta said. IndiGo offered to purchase an electric wheelchair for the child.",Pragya Swastik,2022-05-09T09:50:34.000Z
4,business,India's biggest IPO of LIC subscribed nearly 3 times on final day of bidding,"LIC's IPO, India's biggest IPO which opened on May 4 and closed on May 9, was subscribed 2.95 times on Monday. Expected to raise ₹20,557 crore, the IPO received bids for 47.83 crore equity shares against the IPO size of 16.2 crore shares. The policyholders' portion was subscribed 6.11 times, employees bid 4.39 times and retail investors bid 1.99 times.\n\n",Pragya Swastik,2022-05-09T14:10:38.000Z
...,...,...,...,...,...
94,entertainment,"People taunted me for not getting pregnant, tried IVF: Sambhavna","Actress Sambhavna Seth has revealed that she has been taunted and age-shamed for not being able to get pregnant even after many years of marriage. Sambhavna and her husband Avinash Dwivedi got married in 2016. ""IVF is one of the ways. Some people might know about it, some might not. We have tried it four times and failed,"" Sambhavna said.",Kriti Kambiri,2022-05-09T11:42:26.000Z
95,entertainment,He's done right thing: Neetu on Ranbir not being on social media,"Actress Neetu Kapoor has said that her son Ranbir Kapoor has done the right thing by not being on any social media platforms. Neetu further said that earlier fans had a craze for celebrities and their personal lives and the urge to see them onscreen was high. Now actors post regular updates and have become way more accessible, she added.",Kriti Kambiri,2022-05-09T15:09:18.000Z
96,entertainment,"Nimrat Kaur shares pic of tattoo, says got it for mom with sister","Actress Nimrat Kaur shared a series of pictures on Instagram showing off her wrist tattoo on Sunday to mark Mother's Day. The actress wrote that she got the tattoo along with her sister Ruby five years ago to celebrate their bond with their mother. ""The idea of an infinity knot symbolising Mama...protecting me and Ruby with her love,"" Nimrat wrote.",Kriti Kambiri,2022-05-09T15:23:03.000Z
97,entertainment,Was worried about being written off due to my South Indian accent: Deepika,"Actress Deepika Padukone said that her South Indian accent was frowned upon and that she was initially worried about being written off. ""As an outsider to the i

***

#### 7. Make another dataframe for the Codeup blog posts.      
Name the dataframe codeup_df.

In [2]:
codeup_df = acquire.get_blog_articles()

fetching https://codeup.com/workshops/virtual/learn-to-code-python-workshop-on-4-16/
fetching https://codeup.com/codeup-news/coming-soon-cloud-administration/
fetching https://codeup.com/featured/5-books-every-woman-in-tech-should-read/
fetching https://codeup.com/codeup-news/codeup-start-dates-for-march-2022/
fetching https://codeup.com/codeup-news/vet-tec-funding-dallas/
fetching https://codeup.com/codeup-news/dallas-campus-re-opens-with-new-grant-partner/
fetching https://codeup.com/codeup-news/codeups-placement-team-continues-setting-records/
fetching https://codeup.com/it-training/it-certifications-101/
fetching https://codeup.com/cybersecurity/a-rise-in-cyber-attacks-means-opportunities-for-veterans-in-san-antonio/
fetching https://codeup.com/codeup-news/use-your-gi-bill-benefits-to-land-a-job-in-tech/
fetching https://codeup.com/tips-for-prospective-students/which-program-is-right-for-me-cyber-security-or-systems-engineering/
fetching https://codeup.com/it-training/what-the-heck

In [3]:
codeup_df

title     published  \
0              Learn to Code: Python Workshop on 4/23  Mar 31, 2022   
1                   Coming Soon: Cloud Administration  Mar 17, 2022   
2             5 Books Every Woman In Tech Should Read   Mar 8, 2022   
3                   Codeup Start Dates for March 2022  Jan 26, 2022   
4   VET TEC Funding Now Available For Dallas Veterans   Jan 7, 2022   
5       Dallas Campus Re-opens With New Grant Partner  Dec 30, 2021   
6   Codeup’s Placement Team Continues Setting Records  Nov 19, 2021   
7   IT Certifications 101: Why They Matter, and Wh...  Nov 18, 2021   
8   A rise in cyber attacks means opportunities fo...  Nov 17, 2021   
9    Use your GI Bill® benefits to Land a Job in Tech   Nov 4, 2021   
10  Which program is right for me: Cyber Security ...  Oct 28, 2021   
11               What the Heck is System Engineering?  Oct 21, 2021   
12     From Speech Pathology to Business Intelligence  Oct 18, 2021   
13                      Boris – Behind the Billboards   Oct 3, 2021   
14  Is Codeup the Best Bootcamp in San Antonio…or ...  Sep 16, 2021   

                                              content  
0   According to LinkedIn, the “#1 Most Promising ...  
1   We’re launching a new program out of San Anton...  
2   On this International Women’s Day 2022 we want...  
3   As we approach the end of January we wanted to...  
4   We are so happy to announce that VET TEC benef...  
5   We are happy to announce that our Dallas campu...  
6   Our Placement Team is simply defined as a grou...  
7   AWS, Google, Azure, Red Hat, CompTIA…these are...  
8   In the last few months, the US has experienced...  
9   As the end of military service gets closer, ma...  
10  What IT Career should I choose?\nIf you’re thi...  
11  Codeup offers a 13-week training program: Syst...  
12  By: Alicia Gonzalez\nBefore Codeup, I was a ho...  
13                                                     
14  Looking for the best data science bootcamp in ...

***
#### 8. For each dataframe, produce the following columns:

- title to hold the title
- original to hold the original article/post content
- clean to hold the normalized and tokenized original with the stopwords removed.
- stemmed to hold the stemmed version of the cleaned data.
- lemmatized to hold the lemmatized version of the cleaned data.

***
#### 9. Ask yourself:

- If your corpus is 493KB, would you prefer to use stemmed or lemmatized text?
- If your corpus is 25MB, would you prefer to use stemmed or lemmatized text?
- If your corpus is 200TB of text and you're charged by the megabyte for your hosted computational resources, would you prefer to use stemmed or lemmatized text?

***
***
***
***

Notes from Codeup's Data Science Digital Textbook:
( Most will be copy and pasted ): credit goes to the content creators of the book itself. 
***
## Stemming

Word stems are the base form of a word.

We create new words by attaching affixes in a process known as inflection. For example, "calls", "called", and "calling" all share the base stem "call".

The Porter stemmer is based on the algorithm developed by its inventor, Dr. Martin Porter. Originally, the algorithm is said to have had a total of five different phases for reduction of inflections to their stems, where each phase has its own set of rules.

Note that usually stemming has a fixed set of rules, hence, the root stems may not be lexicographically correct. This means that the stemmed words may not be semantically correct, and might have a chance of not being present in the dictionary (as we'll see in the output of stemming).

***

## Lemmatization

Lemmatization is very similar to stemming, however, the base form in this case is known as the root word, but not the root stem. The difference is that the root word is always a lexicographically correct word (present in the dictionary), but the root stem may not be so. Thus, root word, also known as the lemma, will always be present in the dictionary.

Note that the lemmatization process is considerably slower than stemming, because an additional step is involved where the root form or lemma is formed by removing the affix from the word if and only if the lemma is present in the dictionary.

***

## Removing Stopwords

Words which have little or no significance, especially when constructing meaningful features from text, are known as stop words (or stopwords). These are usually words that end up having the maximum frequency if you do a simple term or word frequency in a corpus. Typically, these can be articles, conjunctions, prepositions and so on. Some examples of stopwords: a, an, the, and like.

While there is no universal stopword list, we will use a standard English language stopwords list from nltk. You can also add your own domain-specific stopwords as needed.

Before removing stopwords, we want to segment text into linguistic units such as words or numbers. This process is called tokenization.